# Artificial Neural Network (ANN) From Scratch
In this notebook I am going to write a complete Neural Network from scratch. To give credit where credit is due, I am going to be relying on Andrew Ng's course for guidance. There are going to be several goals of this project:
1. Neural Networks from scratch. The entire implementation of ANN will be written using numpy and math packages
2. To really understand what Neural Netowrks are about - I am going to write as much intuition behind the code as possible
3. To make this exercise more challenging, I am going to have the structure of a linear part of hidden layer be $X W + b = Y$ (instead of Andrew's $WX + b = Y$)

Final note: to make sure that the implementation is correct I took some test cases from Andrew Ng's course and modified them to comply with our new presentation of a linear part of hidden layer.

## 1. Defining a Structure of our Neural Network
### 1.1 Simplified Intuition
What is a neural network? A neural network is a collection of connected nodes, which loosely remind a structure of a human brain. When receiving an input (a signal) the network passes it to a node, which processes it and passes processed signal to a next neuron. Each time we pass processed input to a new neuron, we learn something new about about our input. This information can then be used to better understand a relationship between an input and an output. Let us assume that we have some observed input and for each input we have some output. Neural Network, can desribe the nature of the relationship between the input and an ouput, learning both linear and non-linear components of that relationship. **Neural Networks help us find linear and non-linear relationship between an input, $X$, and an output, $Y$. $$Y = f(X)$$**

To make things very simple, imagine your input as a single number $x$. When we use this input in our network, we pass $x$ to a neuron, call it $a$. It will $process$ the input, $x$, to, hopefully, produce an insight from that input, which will allow us to learn something of value about our output $y$. Let us call a processed input from a neuron (aka the insight) $a_l$, where $l$ is a layer of our NN, and define it as follows: $a_l = f_l(w_{l} * x + b_{l})$. Let us call the linear transformation step $z_l = w_{l} * x + b_{l}$ - simply doing that will be equivalent to performing a very large OLS (if there are multiple neurons). The advantage of using a neural network is that it allows us to capture not only a linear relatinoship (OLS) between an input and an output, but also a non-linear one. That is why we have $f_l$ function applied to calculating $a_l$. This allows us to approximate a behavior of infinitely complex functions, given, enough of neurons and layers of neurons are used.

### 1.2 General Case
To translate an above simplified explanation to a matrix format, let us first define what an input is, $X$. This matrix is comprised of rows and columns. Where each row is an observation, and each column is a particular feature. Columns can be words and rows can be sentences, columns can be house features (like, distance from center, number of bedrooms, etc.) and rows can be individual houses. 
<img src="/images/X_mat.jpg" style="width:200px;height:200px;">
<caption><center> **Figure 1**</center></caption><br>
What a Neural Netork does is it takes features ($x_1$, $x_2$,..$x_n$ - columns) and those vectors become the input. At each layer of the network, the network 'learns' new patterns from the input, by means of combining various features together - $z_l = w_{l} * x_i + b_{l}$ and applying a non-linear function to detect non-linear patterns: $a_l = f_l(x_i * w_{l} + b_{l})$, where an example of $f_l()$ could be $ReLU$ function. For the general case there can be multiple layers, $l$. Each layer consists of multiple neurons $a_l$'s that are stacked in that layer. To get from layer $l$ to layer $l+1$ in the network, we treat $a_l$'s as inputs and $a_{l+1}$ as our outputs - and repeat the process. THe  intuition behind doing this is as we move up the layers of our Network we learn more complex patterns - details of that relationship.
<img src="images/NN picture.jpg" style="width:600px;height:300px;">
<caption><center> **Figure 2**</center></caption><br>
Note that the dimensions of each layer are not necessarily the same: $n ?= h ?= k ?= l$. That's because it is up to us, the architects of the Neural Network to determine the best structure, depending on the problem.

Now, we have input observations, $X$, and output observations, $Y$. The training of the Neural Network is figuring out the most optimal weights, $w_a$'s above. Those weights are the weights for the various features, that will become new features in the next layer. 

How do we figure out weights? Well, just like a child learning how to walk, our Neural Network will attempt to predict out output with random guess, and will most likely fail. BUT we will gather important information - a measure of difference between our output $Y$, and what our network predicted $\hat{Y}$ - called **the loss**. We can then figure out how to vary the weights, $W_l$'s, such that we minimize the loss. In other words, at each iteration of training we find the derivative of the loss function with respect to each $W_l$ and $b_l$, those derivatives show the direction of greatest increase of the loss function. Therefore, for the next iteration, we are going to take **negative** of that derivative - to make it a direction of steepest descent - and make **some** step (**learning rate**) in that direction.

We figure out those weights by employing **Stochastic Gradient Descent** (SGD) algorithm. SGD is an involved topic, and we can spend a whole new notebook on it. THe main idea of the SGD is that it.

This was a very simple and intuitive overview of Neural Networks. The above is a very crude way to sum up WHAT Neural Networks do and HOW they accomplish it. For each sentence there are blogs, books and tutorials. 

## 1.3 Outline of This Project. Implement L-layer NN
Again the goal is to implement our own Artificial Neural Network in Python. Here is the basic outline:
1. Initialize the parameters
2. Propagation
    1. Compute Forward Propagation
    2. Copmute the Loss Function
    3. Compute Backward Propagation
3. Update parameters

## 2. Coding
### 2.1 Initialize the parameters
To begin with, we are going to initialize our parameters. There is a lot of theory behind how to properly initialize parameters, but the main conclusion is that $b$ can be initialized to a vector of all 0's, but $W$ cannot, otherwise, the NN will not be able to break symmetry and no learning will be done.

Let us next deal with the dimensions of our $W$ and $b$ parameters. For some layer $l$, we have $W_l$, which takes observations in the space of $R_{l-1}$ and transforms them to $R_{l}$. Consider the first step of neural network, input to the first layer:
$$ A1_{m, h}  = X_{m, n}  \times W1_{n, h}  + b_{1, h} $$, where $m$ is the number of observations, $n$ is the number of features in the input layer, h is the number of features in the first layer, $W1$ has dimension $(n, h)$, aka **old dimensions** to **new dimensions**

In [184]:
import numpy as np
import math

from testCases_v4a import * # testing taken from Andrew Ng's course

As input we are given a list , containins the depth of each layer. Thus the length of the list is the total number of layers, and the value of each element is the depth (number of new features, aka h).

In [5]:
def initialize_parameters(layers_list):
    """ 
    input:
        - layer_list - list containing the depth of each layer
    notes:
        - W_l - has dimensions (layers_list[l-1], layers_list[l])
        - b_l - has dimensions (1, layers_list[l])
    """
    np.random.seed(3)
    parameters = {}
    L = len(layers_list)
    
    for l in range(1, L):
        parameters['W_' + str(l)] = np.random.randn(layers_list[l-1], layers_list[l]) * 0.01 # need to multiply by a small number to avoid exploding gradients
        parameters['b_' + str(l)] = np.zeros([1, layers_list[l]])
    
    return parameters

In [6]:
initialize_parameters([5,4,3])

{'W_1': array([[ 0.01788628,  0.0043651 ,  0.00096497, -0.01863493],
        [-0.00277388, -0.00354759, -0.00082741, -0.00627001],
        [-0.00043818, -0.00477218, -0.01313865,  0.00884622],
        [ 0.00881318,  0.01709573,  0.00050034, -0.00404677],
        [-0.0054536 , -0.01546477,  0.00982367, -0.01101068]]),
 'b_1': array([[0., 0., 0., 0.]]),
 'W_2': array([[-0.01185047, -0.0020565 ,  0.01486148],
        [ 0.00236716, -0.01023785, -0.00712993],
        [ 0.00625245, -0.00160513, -0.00768836],
        [-0.00230031,  0.00745056,  0.01976111]]),
 'b_2': array([[0., 0., 0.]])}

### 2.2 Propagation
Now, after we have initialized our parameters we need to propagate forward to calculate our error (loss) and then we need to propagate backward to calculate gradients at each step. We will use those gradients to modify our parameters.

<img src="images/propagation.jpg" style="width:800px;height:240px;">
<caption><center> **Figure 3**</center></caption><br>

#### 2.2.A Forward Propagation
First, we implement forward propagation

Forward Propagation:
1. $Z_l = A_{l-1} \times W_l + b_l$ - this is the linear part of the neuron calculation
2. $A_l = f_l(Z_l)$ - this is the non-linear part of the neuron calculation. $f_l$ most often are ReLU or sigmoid

To correctly implement backward propagation later on, we need to cache some intemediate values for later use. Namely, we need to cache: $Z_l$, $A_{l-1}$, $W_l$, and $b_l$. 

Before we jump into forward propagation, let's quickly define ReLU (Rectified Linear Unit) and sigmoid functions. ReLU takes only the positive part of the array, aka $max(0,Z)$. While sigmoid is this: $\frac{1}{1+e^{-Z}}$

In [7]:
def relu_forward(Z_l):
    """
    Implements a forward activate function ReLU 
    input:
        - Z_l - a linear part of the neuron calculation
    output:
        - A_l - a nonlinear part of the neuron calculation, after applying relu activation function to Z_curr
        - cache - cached value of Z_curr to be used later in the backward propagation step
    """
    A_l = np.maximum(0, Z_l)
    
    return A_l
# print(relu_forward(np.array([ -1,   4,   5, -40])))

def sigmoid_forward(Z_l):
    """
    Implements a forward activate function sigmoid 
    input:
        - Z_l - a linear part of the neuron calculation
    output:
        - A_l - a nonlinear part of the neuron calculation, after applying sigmoid activation function to Z_l
        - cache - cached value of Z_l to be used later in the backward propagation step
    """
    A_l = 1 / (1+ np.exp(-Z_l))
    
    return A_l

# print(sigmoid_forward(np.array([ -1,   4,   5, -40])))

As was outline in Forward Propagation section above, we just need to implement that in code and cache important intermidiate values:

In [8]:
def forward_propagation(A_prev, W_l, b_l, activation = 'relu'):
    """
    Applies a single step of forward propagation, taking A_{l-1} and producing A_l
    
    input:
        - A_prev - previous layer 
        - W_l, b_l - weights for the current to next layer
        - activation - activation function for the layer
    output:
        - A_curr - current layer
        - cache - storing A_prev, W_l, b_l, Z_l for later use in back prop
    """
    
    Z_l = np.dot(A_prev, W_l) + b_l
    
    if activation == 'relu':
        A_curr = relu_forward(Z_l)
    
    if activation == 'sigmoid':
        A_curr = sigmoid_forward(Z_l)
    
    cache = [A_prev, W_l, b_l, Z_l, activation]
    
    return A_curr, cache

In [9]:
A_prev = np.array([[-0.41675785, -2.1361961 , -1.79343559],
       [-0.05626683,  1.64027081, -0.84174737]])
W = np.array([[ 0.50288142],
       [-1.24528809],
       [-1.05795222]])
b = np.array([[-0.90900761]])
forward_propagation(A_prev, W, b, activation = 'sigmoid')

(array([[0.96890023],
        [0.11013289]]),
 [array([[-0.41675785, -2.1361961 , -1.79343559],
         [-0.05626683,  1.64027081, -0.84174737]]),
  array([[ 0.50288142],
         [-1.24528809],
         [-1.05795222]]),
  array([[-0.90900761]]),
  array([[ 3.43896134],
         [-2.08938436]]),
  'sigmoid'])

Note that this is a single forward propagation step, for the full pass through the model we need to loop through the entire structure. This is exactly what we will implement now:

In [10]:
def L_layer_forward(X, parameters):
    """
    Applies forward propagation for the entire network
    input:
        - X - training feature data
        - parameters - all initialized parameters
    output:
        - A_last - last layer activated neurons
        - caches - a list of caches for each layer of the network
    """
    caches = []
    
    L = len(parameters) // 2
    A = X

    for l in range(1, L):
        W = parameters['W_'+str(l)]
        b = parameters['b_'+str(l)]
        A_prev, cache = forward_propagation(A, W, b, activation = 'relu')
        caches.append(cache)
        A = A_prev
    
    W = parameters['W_'+str(L)]
    b = parameters['b_'+str(L)]
    A_L, cache = forward_propagation(A, W, b, activation = 'sigmoid')
    caches.append(cache)

    assert(A_L.shape == (X.shape[0], 1))
    
    return A_L, caches

In [11]:
parameters = initialize_parameters([4, 4, 3, 1])

In [12]:
X = np.array([[-0.31178367,  0.72900392,  0.21782079, -0.8990918 ],
       [-2.48678065,  0.91325152,  1.12706373, -1.51409323],
       [ 1.63929108, -0.4298936 ,  2.63128056,  0.60182225],
       [-0.33588161,  1.23773784,  0.11112817,  0.12915125],
       [ 0.07612761, -0.15512816,  0.63422534,  0.810655  ]])

In [13]:
L_layer_forward(X, parameters)[0]

array([[0.49999998],
       [0.4999998 ],
       [0.50000061],
       [0.5       ],
       [0.50000011]])

In [14]:
X, parameters = L_model_forward_test_case_2hidden()
X = X.T
new_parameters = {}
for i in parameters:
    name = list(i)
    new_parameters[name[0] + '_' + name[1]] = np.transpose(parameters[i])

AL, caches = L_layer_forward(X, new_parameters)

print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.03921668]
 [0.70498921]
 [0.19734387]
 [0.04728177]]
Length of caches list = 3


<table style="width:50%">
  <tr>
    <td> **AL** </td>
    <td > [[ 0.03921668  0.70498921  0.19734387  0.04728177]]</td> 
  </tr>
  <tr>
    <td> **Length of caches list ** </td>
    <td > 3 </td> 
  </tr>
</table>

#### 2.2.B Cost Function
As we mentioned earlier, the goal is to update parameters, based on how the cost function is reacting to changes in parameters. Therefore, defining a cost function is one of thekeys to having a successful Neural Network. There are many choices of such functions, depending on the problem's specifications:
1. If we have a categorization problem, then a good choice can be cross entropy loss, defined as $J = -1/m\sum_{i = 0}^{m}(\sum_{k = 1}^{K}t_i[k]*log(f(s)_i[k]))$, where $t$ is truth and $s$ is the output of the NN, and m is the total number of observations.
2. If we have a regression problem ($Y_{true}$ is continuous), then a good choice can be MSE loss, defined as $J = 1/m*\sum_{i = 1}^{m}(y_i - \hat{y}_i)^2$, where m is the number of observations.

backward propagation's purpose is to obtain parameters' derivatives. 

In [15]:
def compute_cost(A_L, Y, loss = 'categorical'):
    """
    Computes average cost for all training examples using either cross entropy or MSE loss, depending on the specialization
    input:
        - A_L - final layer of the network
        - Y - true y values
        - loss - categorical or continuous
    """
    m = len(Y)
    if loss == 'categorical':
        cost = -1/m * (np.dot(Y.T, np.log(A_L)) + np.dot(1 - Y.T, np.log(1 - A_L)))
    if loss == 'continuous':
        diff = Y - A_L
        cost = 1/m * np.dot(diff.T, diff)^2
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17)
    
    return cost

In [16]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL.T, Y.T)))

cost = 0.2797765635793422


#### 2.2.C Backward Propagation
Now for the last part of this exercise - to build a backward propagation to calculate our derivatives. Remember our visual representation of propagation:
<img src="images/propagation.jpg" style="width:800px;height:240px;">
<caption><center> **Figure 3**</center></caption><br>

Let's approach this first theoretically. Remember that we have the following from our forward propagation step:
1. $Z_l = A_{l-1} \times W_l + b_l$ (Equation 1)
2. $A_l = f_l(Z_l)$, where $f_l$ is some activation function (in our case ReLU or sigmoid)
3. After going through the entire network we also have $J$, a cost function, which is a measure of difference between $A_L$ and $Y$, thus $J \approx A_L$.


We want to find the following derivatives by applying the chain rule:
1. $\frac{\delta J}{\delta W_l} = \frac{\delta J}{\delta A_L} \times \frac{\delta A_L}{\delta A_{L-1}} \times \frac{\delta A_{L-1}}{\delta A_{L-2}} ... \times \frac{\delta A_{l+1}}{\delta A_{l}} \times \frac{\delta A_{l}}{\delta Z_{l}} \times \frac{\delta Z_{l}}{\delta W_{l}} = \frac{\delta J}{\delta Z_l} \times \frac{\delta Z_l}{\delta W_l} = dZ_l \times \frac{\delta Z_l}{\delta W_l} = dZ_l \times A_{l-1}$

What is $\frac{\delta Z_l}{\delta W_l}$ equal to? From Equation 1 we can find the derivative of $Z_l$ with respect to $W_l$:
$\frac{\delta Z_l}{\delta W_l} = \frac{\delta (A_{l-1} \times W_l + b_l)}{\delta W_l} = A_{l-1} $

So we need to understand what $dZ_l$ equal to

2. Now, let's derive $dZ_l = \frac{\delta J}{\delta Z_l} = \frac{\delta J}{\delta A_L} \times \frac{\delta A_L}{\delta A_{L-1}} \times \frac{\delta A_{L-1}}{\delta A_{L-2}} ... \times \frac{\delta A_{l+1}}{\delta A_{l}} \times \frac{\delta A_{l}}{\delta Z_{l}} = \frac{\delta J}{\delta A_l} \times \frac{\delta A_{l}}{\delta Z_{l}} = dA_l * f'_l(Z_l)$ (Equation 2)

Note that $dA_l = \frac{\delta J}{\delta A_l}$, $f'_l(Z_l)$ is a derivative of the activation function in layer $l$ evaluated for $Z_l$, and $*$ is a term-by-term multiplication of matricies.


3. The complicated part from above is $dA_l$. Let's derive it using chain rule: 

$dA_l = \frac{\delta J}{\delta A_l} = \frac{\delta J}{\delta A_L} \times \frac{\delta A_L}{\delta A_{L-1}} \times \frac{\delta A_{L-1}}{\delta A_{L-2}} ... \times \frac{\delta A_{l+1}}{\delta Z_{l+1}} \times \frac{\delta Z_{l+1}}{\delta A_{l}} =  dZ_{l+1} \times \frac{\delta Z_{l+1}}{\delta A_{l}} = dZ_{l+1} \times W_{l+1}$

Note that $\frac{\delta Z_{l}}{\delta A_{l-1}} = W_l$, since $Z_l = A_{l-1} \times W_l + b_l$


Using our newly derived $dA_l$ we substitute it back to Equation 2 to obtain: $dZ_l =  dZ_{l+1} \times W_{l+1} * f'_l(Z_l)$

4. $\frac{\delta J}{\delta b_l} = \frac{\delta J}{\delta A_L} \times \frac{\delta A_L}{\delta A_{L-1}} \times \frac{\delta A_{L-1}}{\delta A_{L-2}} ... \times \frac{\delta A_{l+1}}{\delta A_{l}} \times \frac{\delta A_{l}}{\delta Z_{l}} \times \frac{\delta Z_{l}}{\delta b_{l}} = \frac{\delta J}{\delta Z_l} \times \frac{\delta Z_l}{\delta b_l} = dZ_l \times \frac{\delta Z_l}{\delta b_l} = dZ_l \times 1$, 

where $1$ is the identity matrix since $Z_l = A_{l-1} \times W_l + b_l$

#### 2.2.5.1 Calculation Order
Let's break this up into several steps. We are half way through backward propagation - at layer $l+1$. We have calculated $dA_{l+1}, dW_{l+1}, db_{l+1}, dZ_{l+1}$. We want to get $dA_{l}, dW_{l}, db_{l}, dZ_{l}$:


Function single_backward - propagates through one layer in the network using $dZ_{l+1}$, $W_{l+1}$ to get $dZ_l$ and $dA_l$, utilizing $f'_l(Z_l)$

When we have classification with 2 classes, our final activation function is 'sigmoid' function, which squeeshes activated neurons to be between values of 0 and 1. But for the rest of the layer we can choose either ReLU or 'sigmoid'.

Below is the implementation of the function single_backward. The calculation logic is the same, however, the calculations depend on what activation function was used for that layer. Let's consider two activation functions:
1. ReLU - it is an identity function, when the value is greater or equal to $0$, but when the input is less than $0$, the output is $0$. Thus the derivative is $1$ for values greater than $0$, and $0$ for values that are less than $0$.
2. Sigmoid is usually used as the last activation function for the last layer. If our cost function is cross-entropy and we have a binary classification ($K = 2$), then $J = -1/m\sum_{i = 0}^{m}(y_i*log(AL_i) + (1- y_i)*log(1-AL_i)$. Let us first compute derivative calculation on a single observation and then generalize for all by summing over $m$ and taking the average $\frac{1}{m}$. So consider one example: $J_i = -((y_i*log(AL_i) + (1 - y_i)*log(1-AL_i))$, then $\frac{\delta J_i}{\delta AL_i} = \frac{AL_i - y_i}{AL_i(1-AL_i)}$

So on the case of the binary classification $f_L(Z_L) = A_L = \frac{1}{1+e^{-Z_L}}$, then $f'_L(Z_L) = (\frac{1}{1+e^{-Z_L}})' = ((1+e^{-Z_L})^{-1})' = -1\times-1\times e^{-Z_L}\times \frac{1}{(1+e^{-Z_L})^{-2}} = \frac{e^{-Z_L}}{1+e^{-Z_L}} \times \frac{1}{1+e^{-Z_L}} = (1- \frac{1}{1+e^{-Z_L}})\times \frac{1}{1+e^{-Z_L}} = (1-f_L(Z_L)) \times f_L(Z_L) = (1-A_L) \times A_L$

Thus $\frac{\delta J_i}{\delta ZL_i} = \frac{\delta J_i}{\delta AL_i} \times \frac{\delta AL_i}{\delta ZL_i} = \frac{AL_i - y_i}{AL_i(1-AL_i)}  * f'_L(Z_L) = \frac{AL_i - y_i}{AL_i(1-AL_i)} \times (1-AL_i) \times AL_i = AL_i - y_i$. 

Conclusion is that $dZ_L = \frac{1}{m} \times (A_L - Y)$ for the last layer.

If it is not the last layer, then $dZ_l = dA_l \times (1-A_l) \times A_l$

In [172]:
# cache = [A_prev, W_l, b_l, Z_l, activation]

def single_backward(dA_l, cache):
    """
    Given dA_l, cache - finds the value of dA_prev based on what kind of activation was used.
    input:
        - dA_l is the derivative of J with respect to current hidden layer A_curr
        - cache is a list of A_prev, W_l, b_l, Z_l, activation
    output:
        - dA_prev
    """
    
    A_prev, W_l, b_l, Z_l, activation = cache
    
    if activation == 'relu':
        relu_deriv = np.full(Z_l.shape, fill_value = 1)
        relu_deriv[Z_l <= 0] = 0
        dZ_l = dA_l * relu_deriv
        dA_prev = np.dot(dZ_l, W_l.T) # we only implement 1/m for the L (final layer)
        
    
    if activation == 'sigmoid':
        A_l = sigmoid_forward(Z_l)
        dZ_l = dA_l * (1-A_l) * A_l
        dA_prev = np.dot(dZ_l, W_l.T)

    return dA_prev, dZ_l
    

Now that we have completed a single 'abstract' backward step, we need to put them all into an $L$.

From the get go we are given $A_L$, since that's the output of our forward prop. Next we need to calculate $dA_L$ and $dZ_L$ to pass it to our for loop:

In [173]:
# cache = [A_prev, W_l, b_l, Z_l, activation]

def L_layer_backward(A_L, Y, caches, loss = 'categorical'):
    """
    Performs a full L-layer backward propagation
    inputs:
        - A_L - a final output layer - aka predictions
        - Y - true values the model was predicting
        - caches - contains all caches from forward propagation steps
        - loss - is an indicator what type of loss function we should be using to obtain cost, and first derivative dA_L
    output:
        - grads - at each layer for W and b
    """
    
    L = len(caches)
    m = len(Y)
    Y = Y.reshape(A_L.shape) # after this line, Y is the same shape as A_L
    assert(Y.shape == (m, 1))
    grads = {}
    
    if loss == 'categorical':
        dA_L = 1/m * (-np.divide(Y, A_L) + np.divide(1-Y, 1-A_L))
    if loss == 'continuous':
        dA_L = 1/m * (np.dot(A_L, Y.T))

    cache = caches[L-1] # because caches is a list from 0 to L-1
    A_prev, W_l, b_l, Z_l, activation = cache
    dA_prev, dZ_l = single_backward(dA_L, cache)

    grads['dW_' + str(L)] = np.dot(A_prev.T, dZ_l)
    grads['db_' + str(L)] = np.sum(dZ_l, axis = 0) # same as dZ_1 dot 1

    for l in reversed(range(L-1)): # we have L layers, above we did L, so there are L-1 left to go (0 to L-2)
        A_prev, W_l, b_l, Z_l, activation = caches[l]
        dA_l = dA_prev
        dA_prev, dZ_l = single_backward(dA_l, caches[l])
        grads['dW_' + str(l+1)] = np.dot(A_prev.T, dZ_l)
        grads['db_' + str(l+1)] = np.sum(dZ_l, axis = 0) # same as dZ_l dot 1
        
    
    return grads
    

Testing:

In [174]:
AL, Y_assess, caches = L_model_backward_test_case()
A_L = AL.T
Y = Y_assess.T

# AN's cache is cache = [A_prev, W, b]
# Our cache is cache = [A_prev, W_l, b_l, Z_l, activation]
new_caches = []
new_caches.append([caches[0][0][0].T, caches[0][0][1].T, caches[0][0][2].T, np.dot(caches[0][0][0].T, caches[0][0][1].T) 
             + caches[0][0][2].T, 'relu'])
new_caches.append([caches[1][0][0].T, caches[1][0][1].T, caches[1][0][2].T, np.dot(caches[1][0][0].T, caches[1][0][1].T) 
             + caches[1][0][2].T, 'sigmoid'])
grads = L_layer_backward(A_L, Y, new_caches)
grads

{'dW_2': array([[-0.21308814],
        [-0.12855076],
        [-0.13960917]]),
 'db_2': array([0.15887873]),
 'dW_1': array([[ 0.3127498 , -0.3433267 ,  0.        ],
        [ 0.0580888 , -0.06376802,  0.        ],
        [ 0.10466449, -0.11489732,  0.        ],
        [ 0.07975539, -0.08755291,  0.        ]]),
 'db_1': array([-0.16267208,  0.1785762 ,  0.        ])}

**Expected Output**

<table style="width:60%">
  
  <tr>
    <td > dW1 </td> 
           <td > [[ 0.3127498 , -0.3433267 ,  0.        ]
 [ 0.0580888 , -0.06376802,  0.        ]
 [ 0.10466449, -0.11489732,  0.        ]
 [ 0.07975539, -0.08755291,  0.        ]] </td> 
  </tr> 
  
   <tr>
    <td > db1 </td> 
           <td > [[-0.16267208,  0.1785762 ,  0.        ]] </td> 
  </tr> 
  
</table>



### 2.3 Updating Parameters
Last step in our NN from scratch is to update our parameters: $W$ and $b$! Time to implement the Gradient Descent. Remember that what we did in backward propagation is identify the **direction** in which we should be changing our weights to minimize loss. We still need to know how large of a step we should make in that direction. Here we introduce **learning rate** hyperparameter - controling the magnitude of change.

On the one hand, you want to minimize the loss as fast as possible, and it may be tempting to choose a high learning rate. THe issue is that if you choose it too high - then you may find yourself in a bowl situation. Meaning that you would be swining from one end of the bowl to the next, passing the minimum each time because your step is too big. In some cases, if your step it too high, then your can actually diverge away from the minimum. This hyperparameter often times needs a lot of tuning.

In [177]:
def update_parameters(parameters, grads, learning_rate):
    """
    Updates parameters of our Neural Network utilizing gradients found from the backpropagation step as direction of change
    and learning_rate as a hyperparameter as the magnitude of change. Outputs an updates set of weights - ready to be used
    in the next iteration of learning in our Neural Network.
    
    input
        - parameters - weights, W's, and bias, b's, for each layer l
        - grads - gradients found in backpropagation step indicating direction of change
        - learning_rate - hyperparameter indicating the magnitude of change
    output
        - new_parameters - updated weights and bias terms for each layer l
    """
    
    L = len(parameters) // 2
    new_parameters = {}
    
    for l in range(L):
        new_parameters['W_' + str(l+1)] = parameters['W_' + str(l+1)] - learning_rate * grads['dW_' + str(l+1)]
        new_parameters['b_' + str(l+1)] = parameters['b_' + str(l+1)] - learning_rate * grads['db_' + str(l+1)]
    
    return new_parameters

In [183]:
parameters, grads = update_parameters_test_case()
params_ = {}
for parameter in parameters:
    params_[parameter[0] + '_' + parameter[1]] = parameters[parameter]
grads_ = {}
for grad in grads:
    grads_[grad[:2] + '_' + grad[2]] = grads[grad]

parameters = update_parameters(params_, grads_, 0.1)

print ("W_1 = "+ str(parameters["W_1"]))
print ("b_1 = "+ str(parameters["b_1"]))
print ("W_2 = "+ str(parameters["W_2"]))
print ("b_2 = "+ str(parameters["b_2"]))

W_1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b_1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W_2 = [[-0.55569196  0.0354055   1.32964895]]
b_2 = [[-0.84610769]]


**Expected Output**:

<table style="width:100%"> 
    <tr>
    <td > W1 </td> 
           <td > [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]] </td> 
  </tr> 
  
   <tr>
    <td > b1 </td> 
           <td > [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]] </td> 
  </tr> 
  <tr>
    <td > W2 </td> 
           <td > [[-0.55569196  0.0354055   1.32964895]]</td> 
  </tr> 
  
    <tr>
    <td > b2 </td> 
           <td > [[-0.84610769]] </td> 
  </tr> 
</table>
